In [93]:
import sqlalchemy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import datetime as dt
import seaborn as sns

In [2]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [3]:
inspector = inspect(engine)
inspector.get_table_names()

['measurements', 'station']

In [4]:
inspector.get_columns('measurements')

[{'autoincrement': 'auto',
  'default': None,
  'name': 'id',
  'nullable': False,
  'primary_key': 1,
  'type': INTEGER()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'station',
  'nullable': True,
  'primary_key': 0,
  'type': TEXT()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'date',
  'nullable': True,
  'primary_key': 0,
  'type': TEXT()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'prcp',
  'nullable': True,
  'primary_key': 0,
  'type': INTEGER()},
 {'autoincrement': 'auto',
  'default': None,
  'name': 'tobs',
  'nullable': True,
  'primary_key': 0,
  'type': INTEGER()}]

In [77]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Measurements = Base.classes.measurements
Stations = Base.classes.station

In [20]:
session = Session(engine)
    

## Preciptation Analysis 

In [129]:
# prcp_analysis= session.query(Measurements.date, Measurements.prcp).\
#         filter(func.strftime("%Y", Measurements.date) == "2016").\
#         order_by(Measurements.tobs).all()

# prcp_df = pd.DataFrame(prcp_analysis)
# date_prcp = prcp_df.set_index(prcp_df["date"])
# date_prcp.head()

#x = prcp_df["date"]
#y = prcp_df["prcp"]


precip_2016 =session.query(Measurements.date,Measurements.prcp).\
filter(Measurements.date.between('2016-01-01','2016-12-31')).all()

prec_df= pd.DataFrame(precip_2016)
prec_df.head()

,date,prcp
0,2016-01-01,0.0
1,2016-01-02,0.0
2,2016-01-03,0.0
3,2016-01-04,0.0
4,2016-01-05,0.0


In [142]:
prec_df['date'] =  pd.to_datetime(prec_df['date'])
prec_df.set_index(prec_df['date'], inplace = True)

prec_df.head()

,date,prcp
date,,
2016-01-01,2016-01-01,0.0
2016-01-02,2016-01-02,0.0
2016-01-03,2016-01-03,0.0
2016-01-04,2016-01-04,0.0
2016-01-05,2016-01-05,0.0


In [131]:
prec_df.describe().T

,count,mean,std,min,25%,50%,75%,max
prcp,2069.0,0.179845,0.506363,0.0,0.0,0.02,0.15,9.64


# Station Analysis

In [31]:
session.query(Stations.station).all()

[('USC00519397'),
 ('USC00513117'),
 ('USC00514830'),
 ('USC00517948'),
 ('USC00518838'),
 ('USC00519523'),
 ('USC00519281'),
 ('USC00511918'),
 ('USC00516128')]

In [105]:
# session.query(Stations.station, func.count())\
#     order_by(func.count(Stations.station))\
#     join(Measurements)\
#     order_by(Stations.station).all()


active_stations = session.query(Measurements.station,func.count(Measurements.station))\
.order_by(func.count(Measurements.station))\
.group_by(Measurements.station).all()
    
active_stations

[('USC00518838', 342),
 ('USC00517948', 683),
 ('USC00511918', 1932),
 ('USC00514830', 1937),
 ('USC00516128', 2484),
 ('USC00519523', 2572),
 ('USC00519397', 2685),
 ('USC00513117', 2696),
 ('USC00519281', 2772)]

In [92]:
station_list = []
active_list = [] 

for station, activity in active_stations:
    station_list.append(station)
    active_list.append(activity)
    
station_activity_df = pd.DataFrame({'Activity Count': active_list,'Station_id': station_list})


station_activity_df = station_activity_df[[ 'Station_id','Activity Count']]

station_activity_df()

,Station_id,Activity Count
0,USC00518838,342
1,USC00517948,683
2,USC00511918,1932
3,USC00514830,1937
4,USC00516128,2484
5,USC00519523,2572
6,USC00519397,2685
7,USC00513117,2696
8,USC00519281,2772


In [122]:
active_stations1 = session.query(Measurements.tobs,Measurements.station).\
   group_by(Measurements.station).\
   filter(func.strftime("%Y", Measurements.date) == "2016").\
   order_by(func.max(Measurements.tobs).desc()).all()
active_stations1    

[(84, 'USC00519523'),
 (83, 'USC00514830'),
 (82, 'USC00517948'),
 (82, 'USC00519281'),
 (82, 'USC00519397'),
 (81, 'USC00516128'),
 (80, 'USC00513117')]

In [126]:
Stationhigh_obervations = session.query(Measurements.date,Measurements.tobs,Measurements.station).\
filter(Measurements.date.between('2016-01-01','2016-12-31')).filter_by(station='USC00519523')\
.all()
#USC00519281

Stationhigh_obervations

[('2016-01-01', 64, 'USC00519523'),
 ('2016-01-02', 73, 'USC00519523'),
 ('2016-01-03', 69, 'USC00519523'),
 ('2016-01-06', 74, 'USC00519523'),
 ('2016-01-07', 68, 'USC00519523'),
 ('2016-01-08', 65, 'USC00519523'),
 ('2016-01-09', 77, 'USC00519523'),
 ('2016-01-10', 71, 'USC00519523'),
 ('2016-01-11', 66, 'USC00519523'),
 ('2016-01-12', 68, 'USC00519523'),
 ('2016-01-13', 64, 'USC00519523'),
 ('2016-01-14', 62, 'USC00519523'),
 ('2016-01-15', 67, 'USC00519523'),
 ('2016-01-16', 69, 'USC00519523'),
 ('2016-01-17', 67, 'USC00519523'),
 ('2016-01-18', 71, 'USC00519523'),
 ('2016-01-21', 70, 'USC00519523'),
 ('2016-01-22', 67, 'USC00519523'),
 ('2016-01-25', 70, 'USC00519523'),
 ('2016-01-26', 67, 'USC00519523'),
 ('2016-01-31', 73, 'USC00519523'),
 ('2016-02-01', 69, 'USC00519523'),
 ('2016-02-02', 70, 'USC00519523'),
 ('2016-02-03', 73, 'USC00519523'),
 ('2016-02-04', 70, 'USC00519523'),
 ('2016-02-05', 69, 'USC00519523'),
 ('2016-02-08', 61, 'USC00519523'),
 ('2016-02-09', 59, 'USC0051

In [ ]:
date_list = []
temp_list = []
station_list = []

for date, temp, station 

## Temperature Analysis

In [144]:
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurements.tobs), func.avg(Measurements.tobs), func.max(Measurements.tobs)).\
        filter(Measurements.date >= start_date).filter(Measurements.date <= end_date).all()
print(calc_temps('2016-01-01', '2016-12-31'))

[(56, 74.08554857419043, 84)]


In [150]:
temp_plot = calc_temps('2016-01-01', '2016-12-31')

df_temp_2016 = pd.Dataframe(temp_plot)


AttributeError: module 'pandas' has no attribute 'Dataframe'